# Import Packages

In [485]:
import tabula
from re import sub
import pandas as pd
import numpy as np
import os

# Import Data

In [486]:
pdf_path=f"{os.getcwd()}/discover_10K_2021.pdf"
pdf_data=tabula.read_pdf(pdf_path, pages=61)[0]

In [418]:
pdf_data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,2021,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2020,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,2019,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,Average,NaN,NaN,NaN,NaN,Average,NaN,NaN,NaN,NaN,Average,NaN,NaN,NaN,NaN
1,NaN,NaN,Balance,Yield/Rate,NaN,Interest,NaN,Balance,Yield/Rate,NaN,Interest,NaN,Balance,Yield/Rate,NaN,NaN,Interest
2,Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Interest-earning assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cash and cash equivalents,$,"14,236",0.13,%,$ 18,$,"11,348",0.30,%,$ 35,$,"9,667",2.27,%,$,219
5,Restricted cash,NaN,695,0.03,%,NM,NaN,438,0.45,%,2,NaN,620,2.24,%,NaN,14
6,Other short-term investments,NaN,176,0.12,%,NM,NaN,"2,677",0.14,%,4,NaN,754,2.66,%,NaN,20
7,Investment securities,NaN,"8,713",2.09,%,182,NaN,"11,431",2.21,%,252,NaN,"7,603",2.35,%,NaN,179
8,Loan receivables(1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Credit card loans(2),NaN,"69,365",12.57,%,"8,717",NaN,"71,447",12.58,%,"8,985",NaN,"72,740",13.32,%,NaN,"9,690"


# Data Engineering

## Extract Column Name from the first two columns

In [487]:
def replace_special_char(input_string:str,char_replacement=" "):
    return sub('[^a-zA-Z0-9 \n\.]', char_replacement, input_string)
def join_string_from_list(input_list:list,char_join=" "):
    return char_join.join(input_list)
def snake_case(input_string:str)->str:
  return '_'.join(
    sub('([A-Z][a-z]+)', r' \1',
    sub('([A-Z]+)', r' \1',
    input_string.replace('-', ' '))).split()).lower()

In [488]:
def extract_col_nm(dataframe:pd.DataFrame):
    return dataframe.iloc[[0,1]].agg(lambda x: list(x.dropna())).apply(lambda x: snake_case(replace_special_char(join_string_from_list(x))))


In [496]:
row_desc_ser=extract_col_nm(pdf_data)
row_desc_ser.replace('', np.nan, inplace=True)
row_desc_ser=row_desc_ser.dropna()
row_desc_ser

Unnamed: 2     average_balance
2021                yield_rate
Unnamed: 4            interest
Unnamed: 6     average_balance
2020                yield_rate
Unnamed: 8            interest
Unnamed: 10    average_balance
2019                yield_rate
Unnamed: 13           interest
dtype: object

In [426]:
# Delete the first two rows
pdf_data=pdf_data.drop([0,1]).reset_index(drop=True)
# Make the position column to index
pdf_data=pdf_data.rename(columns={"Unnamed: 0":"position"})
pdf_data["position"]=pdf_data["position"].apply(lambda x: x.split("(")[0])
pdf_data["position"]=pdf_data["position"].apply(replace_special_char)
pdf_data["position"]=pdf_data["position"].apply(snake_case)
pdf_data=pdf_data.set_index(keys="position",drop=True)
pdf_data

,Unnamed: 1,Unnamed: 2,2021,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2020,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,2019,Unnamed: 11,Unnamed: 12,Unnamed: 13
position,,,,,,,,,,,,,,,,
assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
interest_earning_assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cash_and_cash_equivalents,$,"14,236",0.13,%,$ 18,$,"11,348",0.30,%,$ 35,$,"9,667",2.27,%,$,219
restricted_cash,NaN,695,0.03,%,NM,NaN,438,0.45,%,2,NaN,620,2.24,%,NaN,14
other_short_term_investments,NaN,176,0.12,%,NM,NaN,"2,677",0.14,%,4,NaN,754,2.66,%,NaN,20
investment_securities,NaN,"8,713",2.09,%,182,NaN,"11,431",2.21,%,252,NaN,"7,603",2.35,%,NaN,179
loan_receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
credit_card_loans,NaN,"69,365",12.57,%,"8,717",NaN,"71,447",12.58,%,"8,985",NaN,"72,740",13.32,%,NaN,"9,690"
private_student_loans,NaN,"10,057",7.38,%,742,NaN,"9,890",7.63,%,754,NaN,"9,559",8.54,%,NaN,817


## Find Unit Columns

In [427]:
def find_unit_cols(dataframe:pd.DataFrame):
    li_el_in_cols=dataframe.agg(lambda x: list(set(x.dropna())))
    count_el=li_el_in_cols.apply(len)
    col_singleton=count_el[count_el==1]
    col_singleton=list(col_singleton.keys())
    return li_el_in_cols[col_singleton]

In [428]:
ser_unit_cols=find_unit_cols(pdf_data)
ser_unit_cols

Unnamed: 1     [$]
Unnamed: 3     [%]
Unnamed: 5     [$]
Unnamed: 7     [%]
Unnamed: 9     [$]
Unnamed: 11    [%]
Unnamed: 12    [$]
dtype: object

## Extract Unit from a column 

In [429]:
temp_df=pdf_data

In [430]:
# Remove unit columns

li_col=list(temp_df.columns)
[li_col.remove(col) for col in ser_unit_cols.keys()]
print(li_col)

['Unnamed: 2', '2021', 'Unnamed: 4', 'Unnamed: 6', '2020', 'Unnamed: 8', 'Unnamed: 10', '2019', 'Unnamed: 13']


In [431]:
li_new_col=[f"new_col_{str(li_col.index(col))}" for col in li_col]

In [432]:
def delete_fin_unit(string:str)->dict:
    li_spec_char=["$","%"]

    for char in li_spec_char:
        
        if string.find(char)!=-1:
            
            string_split=string.split(" ")
            ind_spec_char=string_split.index(char)
            string_split.pop(ind_spec_char)
            return dict(
                val=string_split[0],
                unit=char
                )
        else:
            return dict(
                val=string,
                unit="nan"
                )

def get_val_unit(dataframe:pd.DataFrame,col_nm:str)->pd.DataFrame:
    return dataframe[col_nm].astype(str).apply(delete_fin_unit).apply(pd.Series).astype(object).replace("nan",np.nan)



In [433]:
def check_unit(df_val_unit:pd.DataFrame):
    temp_res=df_val_unit["unit"].agg(lambda x: list(set(x.dropna())))
    if len(temp_res) == 1:
        return dict(
            fg_unit=True,
            unit=temp_res[0]
            )
    else:
        return dict(
            fg_unit=False,
            unit=None
            )

In [434]:
for col in li_col:
    
    # get value and unit of the column
    col_val_unit=get_val_unit(dataframe=temp_df,col_nm=col)

    # Add unit column whether we have unit in the unit column
    dict_check_unit=check_unit(col_val_unit)
    
    if dict_check_unit["fg_unit"]:

        # get the index of the column
        idx_col=temp_df.columns.get_loc(col)

        if dict_check_unit["unit"]== "$":
            idx_col=idx_col
        elif dict_check_unit["unit"]== "%":
            idx_col=idx_col+1
        
        temp_df[col]=col_val_unit["val"]
        temp_df.insert(loc=idx_col, column=f'new_col: {col}', value=col_val_unit["unit"])

In [435]:
temp_df.columns

Index(['Unnamed: 1', 'Unnamed: 2', '2021', 'Unnamed: 3', 'new_col: Unnamed: 4',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', '2020', 'Unnamed: 7',
       'new_col: Unnamed: 8', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       '2019', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')

In [436]:
temp_df

,Unnamed: 1,Unnamed: 2,2021,Unnamed: 3,new_col: Unnamed: 4,Unnamed: 4,Unnamed: 5,Unnamed: 6,2020,Unnamed: 7,new_col: Unnamed: 8,Unnamed: 8,Unnamed: 9,Unnamed: 10,2019,Unnamed: 11,Unnamed: 12,Unnamed: 13
position,,,,,,,,,,,,,,,,,,
assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
interest_earning_assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cash_and_cash_equivalents,$,"14,236",0.13,%,$,18,$,"11,348",0.30,%,$,35,$,"9,667",2.27,%,$,219
restricted_cash,NaN,695,0.03,%,NaN,NM,NaN,438,0.45,%,NaN,2,NaN,620,2.24,%,NaN,14
other_short_term_investments,NaN,176,0.12,%,NaN,NM,NaN,"2,677",0.14,%,NaN,4,NaN,754,2.66,%,NaN,20
investment_securities,NaN,"8,713",2.09,%,NaN,182,NaN,"11,431",2.21,%,NaN,252,NaN,"7,603",2.35,%,NaN,179
loan_receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
credit_card_loans,NaN,"69,365",12.57,%,NaN,"8,717",NaN,"71,447",12.58,%,NaN,"8,985",NaN,"72,740",13.32,%,NaN,"9,690"
private_student_loans,NaN,"10,057",7.38,%,NaN,742,NaN,"9,890",7.63,%,NaN,754,NaN,"9,559",8.54,%,NaN,817


In [437]:
ser_unit_cols=find_unit_cols(temp_df)
ser_unit_cols

Unnamed: 1             [$]
Unnamed: 3             [%]
new_col: Unnamed: 4    [$]
Unnamed: 5             [$]
Unnamed: 7             [%]
new_col: Unnamed: 8    [$]
Unnamed: 9             [$]
Unnamed: 11            [%]
Unnamed: 12            [$]
dtype: object

In [440]:
ser_unit_cols.keys()

Index(['Unnamed: 1', 'Unnamed: 3', 'new_col: Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 7', 'new_col: Unnamed: 8', 'Unnamed: 9', 'Unnamed: 11',
       'Unnamed: 12'],
      dtype='object')

In [448]:
dict_unit={}

for col in ser_unit_cols.keys():
    idx=list(temp_df.columns).index(col)
    if ser_unit_cols[col][0]== "$":
        dict_unit[temp_df.columns[idx+1]]="$"
    elif ser_unit_cols[col][0]== "%":
        dict_unit[temp_df.columns[idx-1]]="%"


temp_df=temp_df.drop(columns=list(ser_unit_cols.keys()))
temp_df

,Unnamed: 2,2021,Unnamed: 4,Unnamed: 6,2020,Unnamed: 8,Unnamed: 10,2019,Unnamed: 13
position,,,,,,,,,
assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
interest_earning_assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cash_and_cash_equivalents,"14,236",0.13,18,"11,348",0.30,35,"9,667",2.27,219
restricted_cash,695,0.03,NM,438,0.45,2,620,2.24,14
other_short_term_investments,176,0.12,NM,"2,677",0.14,4,754,2.66,20
investment_securities,"8,713",2.09,182,"11,431",2.21,252,"7,603",2.35,179
loan_receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
credit_card_loans,"69,365",12.57,"8,717","71,447",12.58,"8,985","72,740",13.32,"9,690"
private_student_loans,"10,057",7.38,742,"9,890",7.63,754,"9,559",8.54,817


In [450]:
dict_unit

{'Unnamed: 2': '$',
 '2021': '%',
 'Unnamed: 4': '$',
 'Unnamed: 6': '$',
 '2020': '%',
 'Unnamed: 8': '$',
 'Unnamed: 10': '$',
 '2019': '%',
 'Unnamed: 13': '$'}

In [468]:
temp_df=temp_df.dropna(how="all")

In [480]:
def extract_year(dataframe:pd.DataFrame):

    for col in dataframe.columns:
        if col.isdigit():
            return col
    
    # If no year column
    return None

def separate_df(dataframe:pd.DataFrame,num_cols_group:int):
    num_cols=len(dataframe.columns)
    li_dict_df=[]

    for idx in range(0,num_cols,num_cols_group):
        df_separated=dataframe[dataframe.columns[idx:idx+num_cols_group]]
        li_dict_df.append(
            dict(
                year=extract_year(df_separated),
                df=df_separated
            )
        )

    return li_dict_df

In [481]:
li_dict_df=separate_df(temp_df,3)

In [497]:
row_desc_ser

Unnamed: 2     average_balance
2021                yield_rate
Unnamed: 4            interest
Unnamed: 6     average_balance
2020                yield_rate
Unnamed: 8            interest
Unnamed: 10    average_balance
2019                yield_rate
Unnamed: 13           interest
dtype: object

In [502]:
def assign_col_nms_from_desc(dataframe:pd.DataFrame,dict_col_trans=row_desc_ser)->dict:

    dict_col_nm_map={}
    for col_nm in dataframe:
        if col_nm in dict_col_trans.keys():
            dict_col_nm_map[col_nm]=dict_col_trans[col_nm]

    return dict_col_nm_map


In [503]:
li_dict_df=[
    dict(
        year=dict_df["year"],
        df=dict_df["df"].rename(
            columns=assign_col_nms_from_desc(
                dict_df["df"],
                row_desc_ser
            )
        )
    )

    for dict_df in li_dict_df
]

In [506]:
li_dict_df[0]["df"]

,average_balance,yield_rate,interest
position,,,
cash_and_cash_equivalents,"14,236",0.13,18
restricted_cash,695,0.03,NM
other_short_term_investments,176,0.12,NM
investment_securities,"8,713",2.09,182
credit_card_loans,"69,365",12.57,"8,717"
private_student_loans,"10,057",7.38,742
personal_loans,"6,945",12.64,878
other,"2,054",5.57,114
total_loan_receivables,"88,421",11.82,"10,451"
